# Data Deduplication using Clustering
**Objective**: Learn and implement data deduplication techniques.

**Task**: Deduplication Using K-means Clustering

**Steps**:
1. Data Set: Download a dataset containing duplicate customer records.
2. Preprocess: Standardize the data to ensure better clustering.
3. Apply K-means: Use K-means clustering to find and group similar customer records.
4. Identify Duplicates: Identify and remove duplicates within clusters.

In [2]:
import pandas as pd
import numpy as np

# Create a synthetic dataset
data = {
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Hannah'],
    'email': ['alice@example.com', 'bob@example.com', 'charlie@example.com', 'david@example.com', 'eve@example.com', 'frank@example.com', 'grace@example.com', 'hannah@example.com'],
    'age': [25, 30, 35, 40, 28, 33, 38, 41],
    'zip_code': [12345, 12345, 23456, 23456, 34567, 34567, 45678, 45678],
    'purchase_history': [500, 1500, 1200, 800, 1100, 950, 2000, 1800]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save it as CSV to test the code
df.to_csv('customers.csv', index=False)

# Now you can run the deduplication code using this file.
